In [2]:
spark

In [3]:
sc

In [4]:
from pyspark.sql import Row
from datetime import datetime

# Define functions to parse txt files containing itmes, stores, customers, and transactions
def parseStore(s):
    l=s.split('|')
    return Row(store_num=int(l[0]), 
               store_name=l[1],               
               store_zone=l[2],
               store_city=l[3], 
               store_state=l[4], 
               store_type=int(l[5]))

def parseItem(s):
    l=s.split('|')
    return Row(item_number=int(l[0]), 
               dept_categ_class=l[1],               
               item_des=l[2],
               item_unt_qty=float(l[3]), 
               size_unit_desc=l[4], 
               brand_code=l[5], 
               dept_num=int(l[6]), 
               dept_name=l[7], 
               categ_num=int(l[8]), 
               categ_name=l[9], 
               class_num=int(l[10]),
               class_name=l[11])

def parseCustomer(s):
    l=s.split('|')
    return Row(hshld_acct=int(l[0]),
               birth_yr_head_hh=l[1],
               hh_income=l[2],
               hh_size=l[3],
               adult_count=l[4],
               child_count=l[5],
               birth_yr_oldest=l[6],
               birth_yr_youngest=l[7],
               bad_address=l[8],
               privacy=l[9],
               application_date=datetime.strptime(l[10],'%Y-%m-%d'),
               wine_email_sent=int(l[11]),
               wine_email_open=int(l[12]),
               wine_email_click=int(l[13]))

def parsePostrans(s):
    l=s.split('|')
    return Row(hshld_acct=int(l[0]),
               acct_num=int(l[1]),
               trans_num=int(l[2]),
               trans_date=datetime.strptime(l[3],'%Y-%m-%d'),
               store_num=int(l[4]),
               item_number=int(l[5]),
               dept_categ_class=l[6],
               unit_count=int(l[7]),
               net_sales=float(l[8]),
               gross_sales=float(l[9]),
               manuf_coupon=float(l[10]))

In [5]:
path='/public/tbiswas2/csc261/spark/wegmans/'

storeDF=spark.createDataFrame(sc.textFile(path+'wegmans_store_master.txt').map(lambda x: parseStore(x)))
customerDF=spark.createDataFrame(sc.textFile(path+'wegmans_customer_master.txt').map(lambda x: parseCustomer(x)))
itemDF=spark.createDataFrame(sc.textFile(path+'wegmans_item_master.txt').map(lambda x: parseItem(x)))
postransDF=spark.createDataFrame(sc.textFile(path+'partial_transaction.dat').map(lambda x: parsePostrans(x)))

In [5]:
storeDF.show(5)
customerDF.show(5)
itemDF.show(5)
postransDF.show(5)

+-------------+--------------------+---------+-----------+----------+----------+
|   store_city|          store_name|store_num|store_state|store_type|store_zone|
+-------------+--------------------+---------+-----------+----------+----------+
|        DEPEW|   WEGMANS DICK ROAD|       80|         NY|         1|   BUFFALO|
|      AMHERST|WEGMANS ALBERTA D...|       82|         NY|         1|   BUFFALO|
|WILLIAMSVILLE|WEGMANS SHERIDAN ...|       83|         NY|         1|   BUFFALO|
|      BUFFALO|    WEGMANS MCKINLEY|       84|         NY|         1|   BUFFALO|
|      AMHERST|WEGMANS NIAGARA F...|       86|         NY|         1|   BUFFALO|
+-------------+--------------------+---------+-----------+----------+----------+
only showing top 5 rows

+-----------+--------------------+-----------+----------------+---------------+-----------------+-----------+---------+-------+----------+-------+----------------+---------------+---------------+
|adult_count|    application_date|bad_address|birt

# Query 1

In [6]:
storeDF.count()

94

# Query 2

In [7]:
itemDF.select('dept_num').distinct().count()

46

In [8]:
storeDF.groupBy('store_state').count().show()

+-----------+-----+
|store_state|count|
+-----------+-----+
|         NJ|   12|
|         VA|    6|
|         MD|    8|
|         PA|   16|
|         NY|   50|
|         MA|    2|
+-----------+-----+



# Query 3

In [9]:
storeDF.filter(storeDF.store_state == 'NY').count()

50

# Query 4

In [5]:
postransDD = postransDF.select("trans_date").rdd
maxdate = postransDD.max()[0]
mindate = postransDD.min()[0]
print(maxdate)
print(mindate)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 35298)
----------------------------------------


Traceback (most recent call last):
  File "/software/python/2.7.10/b1/lib/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/software/python/2.7.10/b1/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/software/python/2.7.10/b1/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/software/python/2.7.10/b1/lib/python2.7/SocketServer.py", line 655, in __init__
    self.handle()
  File "/software/spark/2.0.0/b1/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/software/spark/2.0.0/b1/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44895)

# Query 5

In [5]:
storeRestrict = storeDF.filter(storeDF.store_name == 'WEGMANS MARKETPLACE')
postransRestrict = postransDF.select('store_num','trans_num')
newtable = postransRestrict.join(storeRestrict, storeRestrict.store_num == postransRestrict.store_num)
newtable1 = newtable.select('trans_num')
newtable1.count()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50160)
----------------------------------------


Traceback (most recent call last):
  File "/software/python/2.7.10/b1/lib/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/software/python/2.7.10/b1/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/software/python/2.7.10/b1/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/software/python/2.7.10/b1/lib/python2.7/SocketServer.py", line 655, in __init__
    self.handle()
  File "/software/spark/2.0.0/b1/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/software/spark/2.0.0/b1/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40965)

# Query 6

In [ ]:
tff=postransDF.agg({"unit_count": "avg"})
tff.show(1)

In [ ]:
from pyspark.sql.functions import col, avg
tf1=postransDF.agg(avg(col("unit_count")))
tf1.show(50)

In [ ]:
postransDF.show(50)

# Query 7

In [6]:
itemDF.filter(itemDF['class_name'] == 'WHOLE MILK').count()

7

# Query 8

In [ ]:
import pyspark.sql.functions as F


milktableDF = itemDF.filter(itemDF['class_name'] == 'WHOLE MILK')
milkfilterDF = milktableDF.select('class_name', 'item_number')
milkjoinDF = postransDF.join(milkfilterDF, milkfilterDF.item_number == postransDF.item_number)
milkhouse = milkjoinDF.select('hshld_acct','unit_count')
milksumDF = milkhouse.groupBy('hshld_acct').agg(F.sum('unit_count').alias('Units_sold'))
maxmilkDD = milksumDF.select('Units_sold').rdd
maxmilkval = maxmilkDD.max()[0]
#print(maxmilkval)
milksumordered = milksumDF.orderBy(milksumDF.Units_sold.desc())
maxhousemilkDF = milksumDF.filter(milksumDF['Units_sold'] == maxmilkval)
maxmilkhshldDF = maxhousemilkDF.select('hshld_acct')
maxmilkhshldnum = maxmilkhshldDF.rdd.max()[0]
milkbuyertuple = customerDF.filter(customerDF['hshld_acct'] == maxmilkhshldnum).show()



# Query 9

In [ ]:
itemDFLB = itemDF.filter(itemDF.size_unit_desc == 'LB')
itemRDD = itemDFLB.select("item_unt_qty").rdd
maxweight = itemRDD.max()[0]
itemDFLBmax = itemDFLB.filter(itemDFLB.item_unt_qty == maxweight)
itemDFLBmax.sort('item_unt_qty',ascending=False).show()

# Query 10

In [ ]:
q10=postransDF.groupBy("trans_num").sum("unit_count")
q10.show(50)

# Query 11

In [ ]:
from pyspark.sql import functions as F
postransDF.join(customerDF, 'hshld_acct') \
    .filter(customerDF.hshld_acct>0) \
    .groupBy('store_num') \
    .agg(F.avg('hh_size').alias('average household size')) \
    .join(storeDF, 'store_num') \
    .select('store_name','average household size') \
    .sort('average household size',ascending=False) \
    .show()

# Query 12

In [ ]:
##Find the top three (3) stores with the highest number of customers. Display the name of the stores and the
##number of customers.

from pyspark.sql import functions as F

distinctposDF = postransDF.select('hshld_acct', 'store_num').distinct()
custcountbystoreDF = distinctposDF.groupBy('store_num').count().sort('count', ascending=False)
#custcountbystoreDF.show()
storecustDF = storeDF.join(custcountbystoreDF, storeDF.store_num == custcountbystoreDF.store_num).select('store_name', 'count')
storecustDF.sort('count', ascending=False).show(3)

# Query 13

In [ ]:
from pyspark.sql import functions as F

postrans12 = postransDF.select('store_num','net_sales','trans_num','trans_date')
salescount = postransDF.groupBy(['store_num','trans_date'])\
            .agg(F.sum('net_sales').alias('summation'))
saleStore = salescount.groupBy('store_num').max('summation')
saleStore.join(storeDF,'store_num').select('store_name','max(summation)')\
            .sort('max(summation)',ascending=False)\
            .show(3)

# Query 14

In [ ]:
from pyspark.sql import functions as F

transsalesitemsDF = postransDF.select('trans_num', 'net_sales','item_number')
transsalesDF = postransDF.select('trans_num','net_sales')
sumsalestransDF = transsalesDF.groupBy('trans_num').agg(F.sum('net_sales').alias('total_net_sales'))
transitemDF = transsalesitemsDF
sumssaletransDD = sumsalestransDF.sort('total_net_sales',ascending=False).rdd
ss11 = sumssaletransDD.zipWithIndex().filter(lambda (r,i): i < 3).map(lambda x: x[0])
dataframenew = spark.createDataFrame(ss11).select('trans_num')
rddnew1 = postransDF.join(dataframenew,'trans_num').select('trans_num','item_number').rdd
rddnew2 = rddnew1.map(lambda x: (x[0],[ x[1] ])).reduceByKey(lambda x,y: x+y)
rddnew2.collect()